In [2]:
import pandas as pd
import re
import yaml
import sqlparse
import os
import pandas as pd
import requests

In [3]:
def add_repo_root_path():
    import os
    import sys
    repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    if repo_root not in sys.path:
        sys.path.append(repo_root)
        
add_repo_root_path()
from src import generate_knowledge

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', 10) 

#### Read dbs

In [10]:
local_dbt_repo = ''
online_dbt_repo = 'https://github.com/dbt-labs/jaffle-shop'

# Use local repo?
if False:
    repo_path = local_dbt_repo
else:
    repo_path = online_dbt_repo

is_online = generate_knowledge.is_online_repo(repo_path)
_, repo_name = generate_knowledge.extract_owner_and_repo(repo_path)
print(repo_name)

jaffle-shop


In [11]:
dbt_models_df = pd.read_csv('../data/dbt_models_' + repo_name + '.csv')
dbt_project_df = pd.read_csv('../data/dbt_project_' + repo_name + '.csv')

In [12]:
display(dbt_models_df.sample(3))
display(dbt_project_df.sample(3))

,path,name,extension,sql_code,config,materialized,is_snapshot,has_jinja_code,model_category,vertical,...,is_filtered,macros,has_macros,parent_models,is_source_model,source,children_models,is_end_model,model_description,jinja_description
5,models/staging/stg_supplies.sql,stg_supplies.sql,.sql,WITH SOURCE AS\n (SELECT *\n FROM {{ source...,NaN,NaN,False,False,stg,supplies,...,False,"['cents_to_dollars', 'dbt_utils.generate_surro...",True,[],True,['ecom.raw_supplies'],"['order_items', 'supplies']",False,Retrieves supply expense data from the 'raw_su...,NaN
8,models/marts/metricflow_time_spine.sql,metricflow_time_spine.sql,.sql,-- metricflow_time_spine.sql\nWITH days AS (--...,NaN,NaN,False,False,other,metricflow_time_spine,...,False,['dbt_date.get_base_dates'],True,[],False,NaN,[],True,"""Generates a list of dates spanning the last 1...",NaN
13,models/staging/__sources.yml,__sources.yml,.yml,NaN,"\n{{\n config(\n materialized=""table...",table,False,False,sources,sources,...,False,NaN,False,[],False,NaN,[],True,"""Selects customer data from the 'raw_customers...",NaN


,path,name,extension,code,is_seed,is_macro,is_test,packages,description
2,package-lock.yml,package-lock.yml,.yml,packages:\n- package: dbt-labs/dbt_utils\n ve...,False,False,False,NaN,NaN
4,macros/cents_to_dollars.sql,cents_to_dollars.sql,.sql,{# A basic example for a project-wide macro to...,False,True,False,NaN,Converts a column representing cents into doll...
8,jaffle-data/raw_orders.csv,raw_orders.csv,.csv,id ...,False,False,False,NaN,NaN
